### Preprocessing


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data=pd.read_csv("destinations_with_coordinates.csv")
df=pd.DataFrame(data)

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
scaler = MinMaxScaler()
data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']] = scaler.fit_transform(
    data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']]
)
if 'latitude' not in data.columns:
    data['latitude'] = np.nan
if 'longitude' not in data.columns:
    data['longitude'] = np.nan

data.to_csv("destinations_with_coordinates.csv", index=False)

print(data.tail())

     pID                pName  culture  adventure  wildlife  sightseeing  \
512  513       Chadani bridge      0.0        0.0       0.0          0.8   
513  514     Ghoda Ghodi lake      0.0        0.0       0.6          0.8   
514  515         Tikapur Park      0.0        0.0       0.0          0.7   
515  516  Godawari Ram Temple      0.8        0.0       0.0          0.8   
516  517       Karnali Bridge      0.0        0.0       0.0          0.8   

     history                            tags  province   latitude  \
512      0.0                           river         7  28.923935   
513      0.0  lake, wildlife, natural beauty         7        NaN   
514      0.7           park, history, forest         7  28.531705   
515      0.0                   hindu, temple         7  27.601747   
516      0.0                 river, scenery          7  28.640886   

            longitude  
512        80.1150514  
513               NaN  
514  81.1200850088515  
515        85.3236257  
516     

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler



data['tags'] = data['tags'].fillna('')  # Handle missing tags
data['tags_list'] = data['tags'].apply(lambda x: [tag.strip() for tag in x.split(',') if tag.strip()])

all_tags = set(tag for tags in data['tags_list'] for tag in tags)
pd.DataFrame({'tag': list(all_tags)}).to_csv("tags.csv", index=False)

tag_weights = {
    "historical": {"culture": 0.8, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 0.6, "history": 1.0},
    "wildlife": {"culture": 0.0, "adventure": 0.0, "wildlife": 1.0, "sightseeing": 0.3, "history": 0.0},
    "adventure": {"culture": 0.0, "adventure": 1.0, "wildlife": 0.5, "sightseeing": 0.5, "history": 0.0},
    "cultural": {"culture": 1.0, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 0.8, "history": 0.7},
    "scenic": {"culture": 0.2, "adventure": 0.0, "wildlife": 0.0, "sightseeing": 1.0, "history": 0.2},
}

scaler = MinMaxScaler()
data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']] = scaler.fit_transform(
    data[['culture', 'adventure', 'wildlife', 'sightseeing', 'history']]
)




features = ['culture', 'adventure', 'wildlife', 'sightseeing', 'history']
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

### model generation

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

features = ['culture', 'adventure', 'wildlife', 'sightseeing', 'history']

feature_matrix = data[features].values
similarity_matrix = cosine_similarity(feature_matrix)

### API integration

In [8]:
import requests

def get_weather(lat, lon):
    """
    Fetch weather information for a given latitude and longitude using OpenWeatherMap API.
    """
    api_key = "13e12c96cea3ae958db69de1f4bf41bd"  
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            weather_data = response.json()
            return {
                "description": weather_data["weather"][0]["description"],
                "temperature": weather_data["main"]["temp"],
                "humidity": weather_data["main"]["humidity"],
            }
        else:
            print(f"API Error: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None

### recommendor

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_weather(lat, lon):
    return {"description": "Sunny"}

def recommend_destinations_with_weather(input_destination, user_preferences=None, selected_tags=None, top_n=5):
    """
    Recommend destinations based on cosine similarity, user preferences, and tags.
    Add weather information to recommendations.
    """
    features = ["culture", "adventure", "wildlife", "sightseeing", "history"]
    feature_matrix = data[features].values

    if input_destination not in data['pName'].values:
        raise ValueError(f"Destination '{input_destination}' not found in dataset.")
    destination_idx = data[data['pName'] == input_destination].index[0]

    similarity_scores = cosine_similarity(feature_matrix, feature_matrix[destination_idx].reshape(1, -1)).flatten()

    if user_preferences:
        user_vector = np.array([user_preferences.get(feature, 0) for feature in features])
        similarity_scores += cosine_similarity(feature_matrix, user_vector.reshape(1, -1)).flatten()

    if selected_tags:
        tag_weights = {tag: 1 for tag in selected_tags}
        for idx, row in data.iterrows():
            row_tags = row['tags'].split(',')
            for tag in row_tags:
                if tag in tag_weights:
                    similarity_scores[idx] += tag_weights[tag]

    data['similarity'] = similarity_scores
    sorted_data = data.sort_values(by='similarity', ascending=False)

    recommendations = sorted_data.head(top_n)
    recommendations_with_weather = []

    for _, row in recommendations.iterrows():
        destination_info = {
            "pID": row['pID'],
            "pName": row['pName'],
            "tags": row['tags'],
            "similarity": row['similarity'],
        }
        lat, lon = row.get('latitude'), row.get('longitude')
        if pd.notnull(lat) and pd.notnull(lon):
            weather_data = get_weather(lat, lon)
            if weather_data:
                destination_info["weather"] = weather_data.get("description", "Weather info not available")
            else:
                destination_info["weather"] = "Weather info not available"
        else:
            destination_info["weather"] = "No weather data available"

        recommendations_with_weather.append(destination_info)

    return pd.DataFrame(recommendations_with_weather)

input_destination = input("Enter a destination: ")
add_preferences = input("Do you want to add preferences? (yes/no): ").strip().lower()
add_tags = input("Do you want to add tags? (yes/no): ").strip().lower()
user_preferences = None
selected_tags = None

if add_preferences == "yes":
    user_preferences = {
        "culture": int(input("Rate your preference for culture (0-5): ")),
        "adventure": int(input("Rate your preference for adventure (0-5): ")),
        "wildlife": int(input("Rate your preference for wildlife (0-5): ")),
        "sightseeing": int(input("Rate your preference for sightseeing (0-5): ")),
        "history": int(input("Rate your preference for history (0-5): "))
    }

if add_tags == "yes":
    selected_tags = input("Enter tags separated by commas (e.g., adventure,culture): ").split(',')

final_recommendations = recommend_destinations_with_weather(
    input_destination,
    user_preferences=user_preferences,
    selected_tags=selected_tags,
    top_n=5
)

print("\nRecommendations with Weather Info:")
print(final_recommendations[['pName', 'weather']])

Enter a destination:  Jiri
Do you want to add preferences? (yes/no):  n
Do you want to add tags? (yes/no):  yes
Enter tags separated by commas (e.g., adventure,culture):  adventure



Recommendations with Weather Info:
                              pName weather
0              Ranipokhari, chitwan   Sunny
1                     Bishazari Tal   Sunny
2  Rafting and Kayaking in Karnali    Sunny
3                       Tamur river   Sunny
4                              Jiri   Sunny
